In [ ]:
from datetime import timedelta
import pandas as pd

def build_calendar_features(calendar_base: pd.DataFrame,
                            window_days: int = 60,
                            as_of_date: pd.Timestamp | None = None) -> pd.DataFrame:
    """
    Aggregate daily calendar data into per-listing availability features.

    Parameters
    ----------
    calendar_base : DataFrame
        Must contain columns ['listing_id', 'date', 'available_target'] where
        available_target is 1 = available, 0 = not available.
    window_days : int, default 60
        Number of days to look back from `as_of_date`.
    as_of_date : pd.Timestamp or None, default None
        End date of the window. If None, use the max date in calendar_base.

    Returns
    -------
    DataFrame
        One row per listing_id with aggregated features:
        ['listing_id', 'availability_rate', 'n_days', 'n_days_available'].
    """
    # Ensure date is datetime
    calendar_base = calendar_base.copy()
    calendar_base["date"] = pd.to_datetime(calendar_base["date"], errors="coerce")

    if as_of_date is None:
        as_of_date = calendar_base["date"].max()

    min_date = as_of_date - pd.Timedelta(days=window_days)

    # Filter to the chosen time window
    mask = (calendar_base["date"] >= min_date) & (calendar_base["date"] <= as_of_date)
    cal_window = calendar_base.loc[mask]

    # Group by listing and aggregate
    features = (
        cal_window
        .groupby("listing_id")["available_target"]
        .agg(
            availability_rate="mean",      # share of days available
            n_days="size",                 # how many days in window
            n_days_available="sum",        # how many days available
        )
        .reset_index()
    )

    # If a listing somehow has no days, fill with 0 (safeguard)
    features["availability_rate"] = features["availability_rate"].fillna(0)

    return features


# ---- Use the function in your pipeline ----

calendar_features = build_calendar_features(calendar_base, window_days=60)
print("Calendar features shape:", calendar_features.shape)
display(calendar_features.head())

# Join with listings to create a modeling table
model_df = listings.merge(calendar_features, on="listing_id", how="inner")
print("Model dataframe shape:", model_df.shape)
display(model_df.head())
